## Deliverable 2. Create a Customer Travel Destinations Map.

In [29]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [30]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tolaga Bay,NZ,-38.3667,178.3000,59.41,70,73,5.61,broken clouds
1,1,Asau,RO,46.4333,26.4000,30.00,69,100,3.31,overcast clouds
2,2,Beringovskiy,RU,63.0500,179.3167,11.39,96,100,26.75,snow
3,3,Rikitea,PF,-23.1203,-134.9692,78.91,66,29,10.69,scattered clouds
4,4,Barranca,PE,-10.7500,-77.7667,76.19,65,4,12.21,clear sky


In [31]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [32]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df
                    


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Rikitea,PF,-23.1203,-134.9692,78.91,66,29,10.69,scattered clouds
4,4,Barranca,PE,-10.7500,-77.7667,76.19,65,4,12.21,clear sky
7,7,Kapaa,US,22.0752,-159.3190,73.40,73,40,16.11,scattered clouds
9,9,Matara,LK,5.9485,80.5353,75.22,87,1,5.37,clear sky
10,10,Port Alfred,ZA,-33.5906,26.8910,70.00,85,9,4.00,clear sky
...,...,...,...,...,...,...,...,...,...,...
212,212,Dingle,PH,10.9995,122.6711,72.66,90,100,9.28,overcast clouds
213,213,Mudgee,AU,-32.5943,149.5871,66.20,88,40,5.75,scattered clouds
214,214,Ribeira Grande,PT,38.5167,-28.7000,64.40,82,75,13.80,broken clouds
218,218,Ulladulla,AU,-35.3500,150.4667,66.20,100,90,2.48,overcast clouds


In [33]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                78
City                   78
Country                76
Lat                    78
Lng                    78
Max Temp               78
Humidity               78
Cloudiness             78
Wind Speed             78
Current Description    78
dtype: int64

In [46]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(0, how='any', inplace=False)


In [47]:
clean_df.count()

City_ID                76
City                   76
Country                76
Lat                    76
Lng                    76
Max Temp               76
Humidity               76
Cloudiness             76
Wind Speed             76
Current Description    76
dtype: int64

In [48]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,78.91,scattered clouds,-23.1203,-134.9692,
4,Barranca,PE,76.19,clear sky,-10.7500,-77.7667,
7,Kapaa,US,73.40,scattered clouds,22.0752,-159.3190,
9,Matara,LK,75.22,clear sky,5.9485,80.5353,
10,Port Alfred,ZA,70.00,clear sky,-33.5906,26.8910,
11,Atuona,PF,78.42,scattered clouds,-9.8000,-139.0333,
21,Kasungu,MW,67.57,overcast clouds,-13.0333,33.4833,
25,Iranshahr,IR,62.60,clear sky,27.2025,60.6848,
26,Port Elizabeth,ZA,69.80,few clouds,-33.9180,25.5701,
27,Victoria,HK,64.00,few clouds,22.2855,114.1577,


In [53]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [67]:
hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,78.91,scattered clouds,-23.1203,-134.9692,Pension Maro'i
4,Barranca,PE,76.19,clear sky,-10.7500,-77.7667,El Olivar Hospedaje
7,Kapaa,US,73.40,scattered clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
9,Matara,LK,75.22,clear sky,5.9485,80.5353,Amaloh Boutique Resort
10,Port Alfred,ZA,70.00,clear sky,-33.5906,26.8910,The Halyards Hotel
...,...,...,...,...,...,...,...
212,Dingle,PH,72.66,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
213,Mudgee,AU,66.20,scattered clouds,-32.5943,149.5871,Parklands Resort & Conference Centre
214,Ribeira Grande,PT,64.40,broken clouds,38.5167,-28.7000,Quinta da Meia Eira
218,Ulladulla,AU,66.20,overcast clouds,-35.3500,150.4667,Sandpiper Motel


In [83]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]

In [84]:
clean_hotel_df.count()

City                   70
Country                70
Max Temp               70
Current Description    70
Lat                    70
Lng                    70
Hotel Name             70
dtype: int64

In [85]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [86]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [90]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))